In [ ]:
!pip install -q timm
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import timm
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tqdm.auto import tqdm

BASE_PATH = "/content"

In [ ]:
!unzip -q /content/task4.zip
train_df = pd.read_csv(f"{BASE_PATH}/task4/train.csv")
test_df = pd.read_csv(f"{BASE_PATH}/task4/test.csv")
sample_submission = pd.read_csv(f"{BASE_PATH}/sample_submission.csv")

train_df['image'] = train_df['file_name'].apply(lambda x: f"{BASE_PATH}/task4/train/{x}")
test_df['image'] = test_df['file_name'].apply(lambda x: f"{BASE_PATH}/task4/test/{x}")

train_df = train_df.dropna(subset=['label'])
train_df['label'] = train_df['label'].astype(int)

In [ ]:
classes = np.unique(train_df['label'])
weights = compute_class_weight(class_weight='balanced', classes=classes, y=train_df['label'])
class_weights = torch.tensor(weights, dtype=torch.float32)

train_split, val_split = train_test_split(
    train_df, 
    test_size=0.15, 
    stratify=train_df['label'], 
    random_state=42
)

In [ ]:
class GameDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['image']).convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        if 'label' in row:
            return image, torch.tensor(row['label'], dtype=torch.long)
        return image

model_name = 'vit_base_patch16_siglip_224.v2_webli'
model = timm.create_model(model_name, pretrained=True, num_classes=len(classes))

data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

train_ds = GameDataset(train_split, transform=transforms)
val_ds = GameDataset(val_split, transform=transforms)
test_ds = GameDataset(test_df, transform=transforms)

batch_size = 16
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

epochs = 9
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}'):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        
    print(f"Epoch {epoch+1} Loss: {train_loss/len(train_loader):.4f}")

In [ ]:
model.eval()
predictions = []
with torch.no_grad():
    for images in tqdm(test_loader, desc='Predicting'):
        images = images.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        predictions.extend(preds.cpu().numpy())

submission = sample_submission[['id', 'task4']].copy()
submission.loc[submission['id'].isin(test_df['id']), 'task4'] = predictions
submission.to_csv("submission_task4.csv", index=False)
print("submission_task4.csv saved")